In [2]:
p=print

In [3]:
from utillll import session

## Basic information.

#### get_specific_node_with_specific_id

In [26]:

def get_record_with_specific_id(tx, id):
    query = f'''

    MATCH (n)
        WHERE n.user_generate_id_7577777777 = "{id}"
        RETURN n
    '''
    result = tx.run(query)
    return list(result)

# k=session.execute_read(get_record_with_specific_id, id="9b6097ab-f834-4789-a542-ced4f9478cc5")

In [ ]:
def get_specific_node_with_specific_id(id):
    k=session.execute_read(get_record_with_specific_id, id=id)
    nodes = []
    for record in k:
        node = record['n']
        node_dict = dict(node)  # properties as dict
        label_list = list(node.labels)  # labels as list
        # Gather everything into one dict
        nodes.append({
            "properties": node_dict,
            "labels": label_list
        })
    return nodes
# get_specific_node_with_specific_id("9b6097ab-f834-4789-a542-ced4f9478cc5")

[{'properties': {'user_generate_id_7577777777': '9b6097ab-f834-4789-a542-ced4f9478cc5',
   'name': 'projectssss'},
  'labels': ['normalNode588888888']}]

#### get_nodewith_specific_property

In [51]:
def get_nodewith_specific_property(tx, property):
    query = f'''

    MATCH (n)
        WHERE n.`{property}` IS NOT NULL
        RETURN n
    '''
    result = tx.run(query)
    return list(result)
k=session.execute_read(get_nodewith_specific_property, property="ue_location_X")

In [52]:
len(k)

456

#### print_number_of_node_and_number_of_connections

In [1]:
def print_number_of_node_and_number_of_connections(session):
        
    # Get total number of node in the database.
    def get_number_of_nodes():
        result = session.run("MATCH (n) RETURN count(n) as total")
        for record in result:
            print(record["total"])
    get_number_of_nodes()


    # Get number of connections.
    def get_number_of_connections():
        result = session.run("MATCH ()-->() RETURN count(*) as total")
        for record in result:
            print(record["total"])
    get_number_of_connections()

In [4]:
from utillll import session
print_number_of_node_and_number_of_connections(session)

456
475


#### get_every_node

In [ ]:
def get_every_node(tx):
    return list(tx.run(
        '''
        MATCH (n) RETURN n

        '''
    ))
result = session.execute_read(get_every_node)
ppp=result[0]
ppp2=ppp["n"]
type(ppp2)
ppp2.keys()
dict(ppp2)

{'ue_location_X': -1056.2984697348495,
 'ue_location_Z': 535.1348803507144,
 'user_generate_id_7577777777': '9b6097ab-f834-4789-a542-ced4f9478cc5',
 'ue_location_Y': -188.69419245425595,
 'name': 'projectssss'}

In [66]:
ppp2

<Node element_id='4:b2f3fb18-14bf-40fa-aa71-612bac4dc40e:0' labels=frozenset({'githubRepo76666666666666'}) properties={'repoid': 776567113, 'id77777777': 'c03fd423-53c7-4f58-a6bf-16eb494cedd3', 'name': 'everythingallaccount/migrates-to-Electron'}>

In [65]:
ppp2.element_id

'4:b2f3fb18-14bf-40fa-aa71-612bac4dc40e:0'

#### get_all_node_and_their_connections13

In [79]:
def get_all_node_and_their_connections13(session):
    # get_all_node_and_their_connections
    def get_all_node_and_their_connections(session):
        result = session.run("MATCH (n)-[r]->(m) RETURN n, r, m")
        return list(result)

    k= session.execute_read(get_all_node_and_their_connections)
    len(k)
    k2=k[0]
    k2.keys()
    nodes=[]
    nodesid={}
    links=[]
    for i in k:
        n=i["n"]
        m=i["m"]

        NID=dict(n)["user_generate_id_7577777777"]
        Ninternal_id=n.element_id
        MID=dict(m)["user_generate_id_7577777777"]
        Minternal_id=m.element_id
        if NID not in nodesid:
            nodesid[NID]=1
            kkkk=dict(n)
            # kkkk["element_id"]=n.element_id
            nodes.append(kkkk)
        if MID not in nodesid:
            nodesid[MID]=1
            kkkk=dict(m)
            # kkkk["element_id"]=m.element_id
            nodes.append(kkkk)

        links.append({"source":

                          NID,
                      "target":
                            MID,
                      }
                        )


    p(len(nodes))
    p(len(links))

    def get_all_node_and_their_connections2(session):
        result = session.run('''
        MATCH (n)
    WHERE NOT EXISTS ((n)--())
    RETURN n


        ''')
        return list(result)

    k=session.execute_read(get_all_node_and_their_connections2)
    for i in k:
        n=i["n"]
        NID=dict(n)["user_generate_id_7577777777"]

        if NID not in nodesid:
            nodesid[NID]=1
            kkkk=dict(n)
            # kkkk["element_id"]=n.element_id
            nodes.append(kkkk)

    oooo={"nodes":nodes, "links":links}
    len(nodes)
    return oooo


In [80]:
get_all_node_and_their_connections13(session)

448
475


{'nodes': [{'ue_location_X': -1056.2984697348495,
   'ue_location_Z': 535.1348803507144,
   'user_generate_id_7577777777': '9b6097ab-f834-4789-a542-ced4f9478cc5',
   'ue_location_Y': -188.69419245425595,
   'name': 'projectssss'},
  {'ue_location_X': -1797.6342281336429,
   'ue_location_Z': 420.77667476557247,
   'user_generate_id_7577777777': '20df637b-2cee-4507-989f-ad6383551982',
   'ue_location_Y': -1074.8219994899275,
   'name': 'graph Viewer project'},
  {'ue_location_X': -860.1159882137663,
   'ue_location_Z': 1505.5394556241126,
   'user_generate_id_7577777777': 'd751d2e3-2c9b-405c-94b2-9e3364b4a089',
   'ue_location_Y': -860.1177224290459,
   'name': 'qualiflyyyy'},
  {'ue_location_X': -394.4175614257018,
   'ue_location_Z': 1513.5645545953012,
   'user_generate_id_7577777777': '40be53da-ed48-4eb6-a85a-a32ba4ba7e0c',
   'ue_location_Y': 1318.7174292905665,
   'name': 'btoyyyy'},
  {'ue_location_X': -734.9118768552388,
   'ue_location_Z': 369.7346652258207,
   'user_generate_id

In [ ]:

# Get all connections
def get_all_connections():
    result = session.run(""
                         "MATCH ()-[r]->() RETURN r"
                         ""
                         ""
                         ""
                         "")
    return list(result)
r=get_all_connections()
r1=r[0]
r2=r1["r"]


r2.nodes

#### _get_constraints

In [ ]:
def _get_constraints(tx):
        query = "SHOW CONSTRAINTS"
        result = tx.run(query)
        return [record for record in result]
k=session.execute_read(_get_constraints)

## Editing things.

#### update_position_of_all_node_772

In [45]:
def update_position_of_all_node_772(session,d):



    output_data = []

    for item in d:
        # Add a new dictionary to output_data with the flattened structure.
        output_data.append({
            "ID": item["ID"],
            "X": item["unreal_engine_location_728"]["X"],
            "Y": item["unreal_engine_location_728"]["Y"],
            "Z": item["unreal_engine_location_728"]["Z"]
        })


    def update_nodes(tx, data):
        query = """

        UNWIND $data AS item
        MATCH (n {user_generate_id_7577777777: item.ID})
        SET n.ue_location_X = item.X,
            n.ue_location_Y = item.Y,
            n.ue_location_Z = item.Z
        RETURN n.ID, n.ue_location_X, n.ue_location_Y, n.ue_location_Z

        """
        result = tx.run(query, data=data)
        return list(result)


    updated_nodes = session.execute_write(update_nodes, output_data)
    return updated_nodes
    #
    # for i in updated_nodes:
    #     p(i)
d=[
      {
        "ID": "9b6097ab-f834-4789-a542-ced4f9478cc5",
        "unreal_engine_location_728": {
          "X": -1056.2984697348495,
          "Y": -188.69419245425595,
          "Z": 535.1348803507144
        }
      }
    ]
a=update_position_of_all_node_772(session,d)

In [16]:
k

[<Record id=3 name='constraint_4a568bb' type='UNIQUENESS' entityType='NODE' labelsOrTypes=['normalNode588888888'] properties=['user_generate_id_7577777777'] ownedIndex='constraint_4a568bb' propertyType=None>]

## Creating things.

In [ ]:

def _create_constraint(tx, label, property):
    query = f"CREATE CONSTRAINT FOR  (n:{label}) REQUIRE  n.{property} IS UNIQUE"
    tx.run(query)
session.execute_write(_create_constraint, "normalNode588888888", "user_generate_id_7577777777")



In [59]:
import uuid




In [54]:
def create_node_tx(tx, name, id8):
    query = ("CREATE (n:normalNode588888888 {"
             "name: $name, "
             "user_generate_id_7577777777:$id8 }) "
             "RETURN n.user_generate_id_7577777777 AS node_id")
    result = tx.run(query, name=name, id8=id8)
    record = result.single()
    return record["node_id"]

#### create_note_with_generate_id

In [69]:
def create_note_with_generate_id(session, name):
    node_id = session.execute_write(create_node_tx, name, str(uuid.uuid4()))
    return node_id

In [ ]:
def create_node_tx_with_position(tx, name, id8, x, y, z):
    query = ("CREATE (n:normalNode588888888 {"
             "name: $name, "
             "user_generate_id_7577777777: $id8, "
             "ue_location_X: $x, "
             "ue_location_Y: $y, "
             "ue_location_Z: $z}) "
             "RETURN n.user_generate_id_7577777777 AS node_id")
    result = tx.run(query, name=name, id8=id8, x=x, y=y, z=z)
    record = result.single()
    return record["node_id"]

#### create_note_with_provided_position_with_generate_id

In [ ]:
def create_note_with_generate_id_and_position(session, name, x, y, z):
    node_id = session.execute_write(create_node_tx_with_position, name, str(uuid.uuid4()), x, y, z)
    return node_id

In [ ]:
already_exists_id="9b6097ab-f834-4789-a542-ced4f9478cc5"
def testing_constraint(session):
    node_id = session.execute_write(create_node_tx, "1", already_exists_id)
    return node_id
k=testing_constraint(session)


In [70]:
testing_name_777="to_be_deleted"


In [71]:
node_id = create_note_with_generate_id(session, testing_name_777)


In [65]:
testing_id_777

'bf4209a5-8723-4476-a0ae-a9a99fed1c8e'

In [72]:
node_id

'75ffeb58-ee1a-4df1-961e-a4b8a5bb7567'

In [75]:
delete_note_with_specific_id(session, node_id)

1

In [6]:

def create_example_integrates(tx, name):

    def a1():

        # https://neo4j.com/docs/api/python-driver/current/api.html#neo4j.Session.execute_write
        def create_node_tx(tx, name):
            query = ("CREATE (n:NodeExample {"
                     "name: $name, "
                     "id: randomUUID()}) "
                     "RETURN n.id AS node_id")
            result = tx.run(query, name=name)
            record = result.single()
            return record["node_id"]
        node_id = session.execute_write(create_node_tx, "Bob")


    def a2():
        # https://neo4j.com/docs/api/python-driver/current/api.html#auto-commit-transactions
        def create_person(driver, name):
            # default_access_mode defaults to WRITE_ACCESS
            with driver.session(database="neo4j") as session:
                query = ("CREATE (n:NodeExample {name: $name, id: randomUUID()}) "
                         "RETURN n.id AS node_id")
                result = session.run(query, name=name)
                record = result.single()
                return record["node_id"]

    def a3():
        def create_and_return_node(tx, node):
            query = (
                "CREATE (n:Character "
                "{"
                "id: $id, "
                "group: $group, "
                "indexColor: $indexColor, "
                "bckgDimensions: $bckgDimensions, "
                "index: $index"
                "}"
                ")"
                "RETURN n"
            )
            result = tx.run(query,
                            id=node["id"],
                            group=node["group"],
                            indexColor=node["__indexColor"],
                            bckgDimensions=node["__bckgDimensions"],
                            index=node["index"])
            return result.single()[0]  # Return the created node


        node = {
            "id": "Cosette",
            "group": 5,
            "__indexColor": "#5003bc",
            "__bckgDimensions": [73.80274419464577, 24.2995279120152],
            "index": 26
        }

        node_id = session.execute_write(create_and_return_node, node)

    def a4():
        employee_threshold = 10


        def example():
            for i in range(100):
                name = f"Thor{i}"
                org_id = session.execute_write(employ_person_tx, name)
                print(f"User {name} added to organization {org_id}")


        def employ_person_tx(tx, name):
            # Create new Person node with given name, if not exists already
            result = tx.run("""
                MERGE (p:Person {name: $name})
                RETURN p.name AS name
                """, name=name
                            )

            # Obtain most recent organization ID and the number of people linked to it
            result = tx.run("""
                MATCH (o:Organization)
                RETURN o.id AS id, COUNT{(p:Person)-[r:WORKS_FOR]->(o)} AS employees_n
                ORDER BY o.created_date DESC
                LIMIT 1
            """)
            org = result.single()

            if org is not None and org["employees_n"] == 0:
                print("Most recent organization is empty.")

                raise Exception("Most recent organization is empty.")
                # Transaction will roll back -> not even Person is created!
            # If org does not have too many employees, add this Person to that
            if org is not None and org.get("employees_n") < employee_threshold:
                result = tx.run("""
                    MATCH (o:Organization {id: $org_id})
                    MATCH (p:Person {name: $name})
                    MERGE (p)-[r:WORKS_FOR]->(o)
                    RETURN $org_id AS id
                    """, org_id=org["id"], name=name
                                )
                print(f"Added {name} to existing organization {org['id']}")

            # Otherwise, create a new Organization and link Person to it
            else:
                result = tx.run("""
                    MATCH (p:Person {name: $name})
                    CREATE (o:Organization {id: randomuuid(), created_date: datetime()})
                    MERGE (p)-[r:WORKS_FOR]->(o)
                    RETURN o.id AS id
                    """, name=name
                                )
                print(f"Created new organization and added {name} to it")

            # Return the Organization ID to which the new Person ends up in
            return result.single()["id"]




## Removing things

#### remove_all

In [ ]:

def remove_all():
    session.run("MATCH (n) DETACH DELETE n")

remove_all()


#### delete_note_with_specific_id


In [66]:
def delete_note_with_specific_id(tx, id):
    # Query that matches any node with the specific ID, deletes it, and counts the deleted nodes.
    query = f'''
    MATCH (n)
    WHERE n.user_generate_id_7577777777 = "{id}"
    DETACH DELETE n
    RETURN count(n) as deletedCount
    '''
    result = tx.run(query)
    return result.single()[0]  # returns the count of deleted nodes

#### delete_note_with_specific_idand_label

In [34]:

def delete_note_with_specific_idand_label(tx, label, id):
    # Query that matches node with specific label and ID, deletes it, and counts deleted nodes.
    query = f'''
    MATCH (n:{label})
    WHERE n.user_generate_id_7577777777 = "{id}"
    DETACH DELETE n
    RETURN count(n) as deletedCount
    '''
    result = tx.run(query)
    return result.single()[0]  # returns the count of deleted nodes

In [35]:
k=delete_note_with_specific_idand_label(session, "NodeExample", "9b6097ab-f834-4789-a542-ced4f9478cc5")

In [36]:
k

0